In [1]:
import pandas as pd
import numpy as np
import pydicom 
import os
import matplotlib.pyplot as plt

from keras.models import Model, Sequential
from keras.layers import Concatenate, Dense, Input, concatenate
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten

Using TensorFlow backend.


In [2]:
def load_best_slices(path):
    ind = []
    for file in os.listdir(path):
        if file.split('.')[1] == 'txt':
            ind.append(file.split('.')[0])
        
    df = pd.DataFrame(index = ind, columns= ['CT'])
    for ind in df.index:
        df.loc[ind].CT = np.loadtxt(path + ind + '.txt')
    
    return df

df = load_best_slices('best_lung_slice/')

In [3]:
linear_data_all = pd.read_csv('patient_slope_intercept.csv', index_col=0)
result = pd.DataFrame(index = df.index, columns = ['slope'])

for ind in result.index:
    result.loc[ind].slope = linear_data_all.loc[ind].slope

In [4]:
dataset = df.values

In [5]:
X = []
for ind in df.index:
    X.append(df.loc[ind].CT)

X = np.array(X)/10
X = X.reshape(X.shape[0], 512, 512, 1)

In [9]:
from sklearn.preprocessing import MinMaxScaler
y = result.values
scaler = MinMaxScaler()
scaler.fit(y)
y_scaled = scaler.transform(y)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2)

In [15]:
# building a linear stack of layers with the sequential model
model = Sequential()
# convolutional layer
model.add(Conv2D(10, kernel_size=(2,2), strides=(1,1), padding='valid', activation='relu', 
                 input_shape=(512,512,1), kernel_initializer='random_normal', 
                 bias_initializer='zeros' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(10, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='relu'))
# output layer
model.add(Dense(1, activation='sigmoid'))

ValueError: A `Concatenate` layer should be called on a list of at least 2 inputs

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 511, 511, 10)      50        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 255, 255, 10)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 253, 253, 10)      910       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 126, 126, 10)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 126, 126, 10)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 158760)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

In [9]:
model.compile(loss='mean_squared_error', metrics=['mean_squared_error'], optimizer='adam')

In [10]:
model.fit(X_train,y_train, batch_size=32, epochs=20, validation_data=(X_test,y_test))

Train on 140 samples, validate on 36 samples
Epoch 1/20
140/140 [==============================] - 9s 62ms/step - loss: 0.0227 - mean_squared_error: 0.0227 - val_loss: 0.0166 - val_mean_squared_error: 0.0166
Epoch 2/20
140/140 [==============================] - 8s 58ms/step - loss: 0.0210 - mean_squared_error: 0.0210 - val_loss: 0.0178 - val_mean_squared_error: 0.0178
Epoch 3/20
140/140 [==============================] - 8s 59ms/step - loss: 0.0204 - mean_squared_error: 0.0204 - val_loss: 0.0153 - val_mean_squared_error: 0.0153
Epoch 4/20
140/140 [==============================] - 8s 55ms/step - loss: 0.0194 - mean_squared_error: 0.0194 - val_loss: 0.0174 - val_mean_squared_error: 0.0174
Epoch 5/20
140/140 [==============================] - 8s 55ms/step - loss: 0.0184 - mean_squared_error: 0.0184 - val_loss: 0.0161 - val_mean_squared_error: 0.0161
Epoch 6/20
140/140 [==============================] - 8s 56ms/step - loss: 0.0176 - mean_squared_error: 0.0176 - val_loss: 0.0164 - val_mean

In [11]:
scaler.inverse_transform(model.predict(X_test))

array([[-4.3214703],
       [-5.0042615],
       [-6.640951 ],
       [-6.9929423],
       [-5.936095 ],
       [-5.2826657],
       [-2.5023305],
       [-6.5002594],
       [-5.2864013],
       [-5.141029 ],
       [-3.7860622],
       [-7.1335535],
       [-4.7390203],
       [-3.6418474],
       [-5.6583247],
       [-6.1442204],
       [-5.455256 ],
       [-2.5533507],
       [-6.624594 ],
       [-5.854339 ],
       [-2.2301297],
       [-2.6023266],
       [-3.7801142],
       [-5.3650274],
       [-5.90843  ],
       [-1.4496009],
       [-3.7028086],
       [-6.133801 ],
       [-5.5570817],
       [-5.589111 ],
       [-5.2824535],
       [-4.96003  ],
       [-6.3055406],
       [-1.1737467],
       [-3.9894502],
       [-6.650479 ]], dtype=float32)

In [12]:
n=8
example = df.iloc[n].CT.reshape(1,512,512,1)
print(scaler.inverse_transform(model.predict(example)))
print(result.iloc[n])

[[13.25292]]
slope   -3.41031
Name: ID00229637202260254240583, dtype: object


In [13]:
abs(scaler.inverse_transform(model.predict(X_test)) - scaler.inverse_transform(y_test)).mean()

3.7031015758732995

In [14]:
scaler.inverse_transform(y_test)

array([[-10.10365464],
       [ -0.59836422],
       [ -2.01667713],
       [ -0.43741259],
       [ -4.20860361],
       [ -2.21685139],
       [  9.05220839],
       [ -2.90423317],
       [ -4.26350233],
       [ -0.57674104],
       [ -3.44357802],
       [-12.68318624],
       [  0.35006384],
       [ -2.28098694],
       [ -0.0377488 ],
       [ -7.4604788 ],
       [ -6.40337866],
       [ -6.04144002],
       [ -4.54892513],
       [ -2.33530229],
       [ -1.4086339 ],
       [  1.89659994],
       [ -3.0797419 ],
       [ -1.08055576],
       [ 14.68261173],
       [  0.59778705],
       [ -4.44100719],
       [ -4.18569593],
       [-16.46481705],
       [ -4.51304901],
       [ -5.87359531],
       [ -5.49436856],
       [ -2.3159204 ],
       [ -2.83713956],
       [ -5.73331844],
       [ -7.61721992]])